<a href="https://colab.research.google.com/github/neurologic/NeurophysiologyModules/blob/main/EMGMovementLab_DLCTF2_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2+ Toolbox | Colab for TensorFlow 2+
https://github.com/DeepLabCut/DeepLabCut

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1607087333173-TG4DRQR5WZ6ETXAJHXEO/ke17ZwdGBToddI8pDm48kKdgWGJOTWy1pqJZDMO95txZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZUJFbgE-7XRK3dMEBRBhUpwsA4PS9IFCXiEkiYtvZKsVufHa_vZp4Er3aXh9hA8Hg33vCVX1-CYF3ghyfBsW8js/dlcTF.png?format=1500w)

This notebook illustrates how to use colab:
- analyze novel videos
- filter data
- create videos

###This notebook assumes you already have a project folder with a trained network and that the project folder has been uploaded to Google Drive.
###This notebook assumes you have taken videos to analyze that are uploaded to your Google Drive.

NOTE: This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper.

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.

Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"-> and then select "GPU" (also choose "Python 3" if prompted)


In [ ]:
#@title Hit the play button to the left to install DeepLabCut. A LOT of text will appear... and keep appearing. Be patient and wait until it is completely finished.
#Install the branch with tf2.x support:
!pip install git+https://github.com/DeepLabCut/DeepLabCut-core.git@tf2.2alpha
!pip install tf_slim

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

In [ ]:
#@title import software
import deeplabcutcore as deeplabcut

from datetime import datetime
import pandas as pd
import numpy as np

#GUIs don't work on the cloud, so we supress them:
import os
os.environ["DLClight"]="True"

# stifle tensorflow warnings, like we get it already.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# import deeplabcutcore as deeplabcut
# import tensorflow
# tensorflow.__version__

In [ ]:
#@title link to your google drive
from google.colab import drive
drive.mount('/content/drive')

To edit any of the .yaml configuration files in the project, you can simply double click on the file on the left, and edit within colab.


In [ ]:
# PLEASE EDIT THE VIDEOFILE_PATH:
# videofile_path = ['path to your video file'] #Enter the list of videos or folder to analyze.
videofile_path = ['/content/drive/Shareddrives/BIOL247/ClassMovementData_20211122/AF_video2021-11-22T14_20_51.avi']


### YOU SHOULD NOT NEED TO EDIT THE VIDEOTYPE OR PATH_CONFIG_FILE:
#Setup your project variables:
VideoType = '.avi'  # everyone's video type should be the same, since we used the same program)

# path_config_file = 'path to the config.yaml file of the DLC project that you downloaded'
path_config_file = '/content/drive/Shareddrives/BIOL247/EMG_Tracking_ZooInitialized-lab/config.yaml'

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
deeplabcut.analyze_videos(path_config_file, videofile_path, videotype=VideoType)

You can also now filter your data to smooth any small jitters:

In [ ]:
deeplabcut.filterpredictions(path_config_file, videofile_path, videotype=VideoType)

You can look at the table that has the x and y position for every tracked body part in every frame
> The results are stored in hd5 file in the same directory where the video resides. 

In [ ]:
path_to_h5 = 'insert full path to _filtered.h5 file including the file name' # the file name will end in '_filtered.h5'

In [ ]:
#@title import results and display as a table
df = pd.read_hdf(path_to_h5)
df = df[df.keys()[0][0]]
display(df)

## Define the skeleton for your movement.

Define a skeleton based on the "bones" that you want to include in your analysis </br>
*Full list of possible body parts:*
- ankle1
- knee1
- hip1
- hip2
- knee2
- ankle2
- wrist1
- elbow1
- shoulder1
- shoulder2
- elbow2
- wrist2
- chin
- forehead

The format of the skeleton should be as follows...
<code>skeleton = [
     ['body part','body part'],
     ['body part','body part'] 
 ]</code>
 Add more entries as needed

In [ ]:
skeleton = [
    ['wrist1','elbow1'],
    ['elbow1','shoulder1'],
    ['wrist2','elbow2'],
    ['elbow2','shoulder2'],
    ['shoulder1','shoulder2'] # if add more entries after this, terminate this line with a comma
    # add more entries as needed
]

## Run the code below to edit the config file based on your skeleton. You can make any other edits to the parameters listed that you want as well. 

In [ ]:
# edit the configuration file to make these changes
config_path = path_config_file[0]
edits = {'dotsize': 5,
          'colormap': 'jet',
          'pcutoff': 0.1,
          'skeleton': skeleton,
          'skeleton_color': 'red'}
deeplabcut.auxiliaryfunctions.edit_config(path_config_file, edits);

## Create labeled video:
This funtion is for visualization purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
#@title Create a labeled video  
#@markdown This funtion is for visualization purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

deeplabcut.create_labeled_video(path_config_file, videofile_path, videotype=VideoType, filtered=True, draw_skeleton=True,trailpoints=5)

In [ ]:
#@title Analyze the skeleton to get your "bones"
deeplabcut.analyzeskeleton(path_config_file, videofile_path, videotype=VideoType, save_as_csv=True)

The results are stored in another .h5 file

In [ ]:
path_to_h5 = 'insert the full path to the _skeleton.h5 file including the file name' # the file name will end in '_skeleton.h5'

In [ ]:
#@title display results
df = pd.read_hdf(path_to_h5)
display(df)